# Extended ra operations
> relation algebra operations and ie function calculation operations

In [ ]:
#| default_exp ra

In [ ]:
"""
TODO how to solve calc outputs binding to different inputs

 we cant simply join inputs and outputs, because then outputs from different inputs can intermingle

 # we need to keep track of the input index and join inputs and their outputs on free vars plus input index
 # maybe we want to also inline the renaming into the calc node

"""

'\nTODO how to solve calc outputs binding to different inputs\n\n we cant simply join inputs and outputs, because then outputs from different inputs can intermingle\n\n # we need to keep track of the input index and join inputs and their outputs on free vars plus input index\n # maybe we want to also inline the renaming into the calc node\n\n'

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
#| export
import pytest
import pandas as pd
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
import networkx as nx
import itertools

from spannerlib.utils import serialize_df_values
from spannerlib.data_types import _infer_relation_schema,pretty

import logging
logger = logging.getLogger(__name__)

In [ ]:
#| export
def _col_names(length):
    # these names wont conflixt with logical variables since they must always start with Uppercase letters
    return [f'col{i}' for i in range(length)]



In [ ]:
s = pd.DataFrame([
    [1,1],
    [2,2],
    [3,3],
    [4,5]
])

s2 = pd.DataFrame([
    [1,2,3],
    [2,3,4],
    [2,3,5],
    [4,5,6]
])

In [ ]:
df = pd.DataFrame([
    [1,2],
    [1,3],
    [1,2],
    ])
df

,0,1
0,1,2
1,1,3
2,1,2


### RA

In [ ]:
#| export
def select(df,theta,**kwargs):
    if df.empty:
        return df

    if callable(theta):
        return df[theta(df)]
    else:
        pos_val_pairs = theta
        for pos,val in pos_val_pairs:
            df = df[df.iloc[:,pos] == val]
        return df

def project(df,on=None,not_on=None,**kwargs):
    if df.empty:
        return df
    if on is None and not_on is None:
        raise Value("either on or not_on must be specified")
    if on is not None:
        return df[on]
    else:
        return df.drop(columns=not_on)

def rename(df,names,**kwargs):
    if df.empty:
        return df
    names_mapper = {
    }
    current_col_names = list(df.columns)
    for i,name in names:
        names_mapper[current_col_names[i]] = name
    return df.rename(names_mapper,axis=1)

def union(*dfs,**kwargs):
    return pd.concat(dfs).drop_duplicates()

def intersection(df1,df2,**kwargs):
    return pd.merge(df1,df2,how='inner',on=list(df1.columns))

def difference(df1,df2,**kwargs):
    return pd.concat([df1,df2]).drop_duplicates(keep=False)

def join(df1,df2,**kwargs):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    on = cols1 & cols2
    # get only logical variables
    on = [ col for col in on if isinstance(col,str) and col[0].isupper()]
    if len(on)==0:
        return pd.merge(df1,df2,how='cross')
    else:
        return pd.merge(df1,df2,how='inner',on=on)


#### Tests

In [ ]:
s3 = pd.DataFrame([
    [4,5,6],
    [5,6,7],
    [1,2,3],
    [7,8,9]
])
s3

,0,1,2
0,4,5,6
1,5,6,7
2,1,2,3
3,7,8,9


In [ ]:
s

,0,1
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
s2_copy = s2.copy()
s2

,0,1,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
empty = pd.DataFrame()

truth = pd.DataFrame([()])

In [ ]:
empty.empty

True

In [ ]:
assert serialize_df_values(select(empty,[(0,1)]))==set()
assert serialize_df_values(rename(empty,names=[(0,'X')]))==set()
assert serialize_df_values(project(empty,on=['X','Y']))==set()

In [ ]:
res = select(s,[(0,1)])
assert serialize_df_values(res)=={(1,1)}

res = select(s2,[(0,2),(1,3)])
assert serialize_df_values(res) == {(2,3,4),(2,3,5)}

res = select(s,lambda row: row[0] == row[1])
assert serialize_df_values(res) == {(1, 1), (2, 2), (3, 3)}

In [ ]:

s2

,0,1,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
res = project(s2,on=[2,1])
assert serialize_df_values(res) == {(3, 2), (4, 3), (5, 3), (6, 5)}
res = project(s2,not_on=[0,1])
assert serialize_df_values(res)== {(3,), (4,), (5,), (6,)}

In [ ]:
assert list(rename(s2,[(0,'X')]).columns) == ['X',1,2]
assert list(rename(s2,[(0,'X'),(2,'Z')]).columns) == ['X',1,'Z']

In [ ]:
res = union(s2,s3)
assert serialize_df_values(res) == {(1, 2, 3), (2, 3, 4), (2, 3, 5), (4, 5, 6), (5, 6, 7), (7, 8, 9)}

res = intersection(s2,s3) 
assert serialize_df_values(res) == {(1, 2, 3), (4, 5, 6),}

res = difference(s2,s3)
assert serialize_df_values(res) == { (2, 3, 4), (2, 3, 5),(5, 6, 7), (7, 8, 9)}


In [ ]:
left = rename(s,[(1,'Y')])
left

,0,Y
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
right = rename(s2,[(0,'Y'),(1,'X')])
right

,Y,X,2
0,1,2,3
1,2,3,4
2,2,3,5
3,4,5,6


In [ ]:
res = join(left,right)
assert serialize_df_values(res) == {(1, 1, 2, 3), (2, 2, 3, 4), (2, 2, 3, 5)}
res

,0,Y,X,2
0,1,1,2,3
1,2,2,3,4
2,2,2,3,5


In [ ]:
res = join(rename(s,[(0,'a'),(1,'b')]),s)
assert len(res)==16
assert list(res.columns) == ['a', 'b', 0, 1]
res.head()

,a,b,0,1
0,1,1,1,1
1,1,1,2,2
2,1,1,3,3
3,1,1,4,5
4,2,2,1,1


### Calc ie operators

To calculate ie functions we need 2 basic operators
* a map like operator to run the function on the input tuples to generate output tuples
* an index like operator that adds a unique index as the last column to a dataframe
  * this is used to sync input and output tuples when joining
  
The map operator assumes an index as the first column and adds it to the output tuple while removing it from the ie function call

In [ ]:
#| export
def index(df,key_col_name='index',**kwargs):
    # and an "index" operator that adds an uniq index with a key to a given column
    index_series = pd.Series(range(len(df)),name=key_col_name)
    return pd.concat([df,index_series],axis=1)

In [ ]:
#| export
def assert_ie_output_schema(input,output,expected_schema):
    """helper function for asserting the output schema of an ie function
    expected schema is either a callable that takes input and output and 
    raises error if the output is notvalid, or a fixed list of column names

    Args:
        input (_type_): input relation
        output (_type_): output relation from ie function
        expected_schema (_type_): output schema object

    Raises:
        ValueError: _description_
        ValueError: _description_
    """
    if callable(expected_schema):# out schema is a callable
        if not expected_schema(output,input):
            raise ValueError(f"When validating output schema using {expected_schema} for input output pair ({input},{output}), the output was invalid")
    else: # out schema is a fixed list
        if _infer_relation_schema(output) != expected_schema:
            raise ValueError(
                f"IEFunction returned a value of an unexpected schema {pretty(_infer_relation_schema(output))} "
                f"for input output pair ({input},{output})"
                f"expected {pretty(expected_schema)}")


def map_iter(df,func,out_schema,**kwargs):
    """helper function returns schema of output, and then the output rows as an iterator
    """
    first_out_schema = None
    for _,in_row in df.iterrows():
        in_row = list(in_row)
        index = in_row[-1]
        ie_input = in_row[:-1]
        for out_row in func(*ie_input):
            assert_ie_output_schema(ie_input,out_row,out_schema)
            if first_out_schema is None:
                first_out_schema = _infer_relation_schema(out_row)
                yield first_out_schema
            index_out_put = list(out_row) + [index]
            yield index_out_put

def ie_map(df,func,out_schema,index_name ='index',**kwargs):
    """given an indexed dataframe, apply an ie function to each row and return the output 
    such that each output relation is indexed by the same index as the input relation that generated it
    """
    if df.empty:
        return df
    output_iter = map_iter(df,func,out_schema)
    schema = next(output_iter)
    num_cols = len(schema)
    return pd.DataFrame(output_iter,columns=_col_names(num_cols)+[index_name])





#### Tests

In [ ]:
s

,0,1
0,1,1
1,2,2
2,3,3
3,4,5


In [ ]:
assert serialize_df_values(index(s)) == {(1, 1, 0), (2, 2, 1), (3, 3, 2), (4, 5, 3)}
# make sure index doesnt change the original df
assert serialize_df_values(s) == {(1, 1), (2, 2), (3, 3), (4, 5)}

In [ ]:
assert_ie_output_schema((1,2),[1,2],lambda x,y: True)

In [ ]:
def func(x,y): return [(x+y,x-y)]
def func2(x,y,z): return [(x,y)]

res = ie_map(index(s),func,[int,int])
assert serialize_df_values(res) == {(2, 0, 0), (4, 0, 1), (6, 0, 2), (9, -1, 3)}
res = ie_map(index(s),func,lambda x,y: True)
assert serialize_df_values(res) == {(2, 0, 0), (4, 0, 1), (6, 0, 2), (9, -1, 3)}
res

,col0,col1,index
0,2,0,0
1,4,0,1
2,6,0,2
3,9,-1,3


In [ ]:
# test checking of schema
with pytest.raises(ValueError) as exc_info:
    ie_map(index(s2),func2,[int,int,int])
assert 'returned a value of an unexpected schema' in str(exc_info.value)
print(exc_info.value)

with pytest.raises(ValueError) as exc_info:
    ie_map(index(s2),func2,[int,str])
assert 'returned a value of an unexpected schema' in str(exc_info.value)
print(exc_info.value)

IEFunction returned a value of an unexpected schema [<class 'int'>, <class 'int'>] for input output pair ([1, 2, 3],(1, 2))expected [<class 'int'>, <class 'int'>, <class 'int'>]
IEFunction returned a value of an unexpected schema [<class 'int'>, <class 'int'>] for input output pair ([1, 2, 3],(1, 2))expected [<class 'int'>, <class 'str'>]


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     